In [1]:
import models
# import datas
import configs

import argparse
import torch
import torchvision
import torchvision.transforms as TF
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time
import os
from math import log10
import numpy as np
import datetime
from utils.config import Config
from tensorboardX import SummaryWriter
import sys

import time

from imageio import imread
import cv2

In [2]:
# # loading configures
# parser = argparse.ArgumentParser()
# parser.add_argument('config')
# args = parser.parse_args()
# # args = parser.parse_config()

# config = Config.from_file(args.config)

class Config():
    
    def __init__(self):
        self.mean = [0.429, 0.431, 0.397]
        self.std = [1, 1, 1]
        self.testset_root= None
        self.trans = None
        self.test_size = None
        self.test_crop_size = None
        self.inter_frames = 1
        self.model = 'QVI'
        self.pwc_path = './utils/network-default.pytorch'
        self.store_path = 'outputs/example/'
        self.checkpoint = 'checkpoints/quadratic/model.ckpt'

config = Config()

In [3]:


# preparing datasets
normalize1 = TF.Normalize(config.mean, [1.0, 1.0, 1.0])
normalize2 = TF.Normalize([0, 0, 0], config.std)
trans = TF.Compose([TF.ToTensor(), normalize1, normalize2, ])

revmean = [-x for x in config.mean]
revstd = [1.0 / x for x in config.std]
revnormalize1 = TF.Normalize([0.0, 0.0, 0.0], revstd)
revnormalize2 = TF.Normalize(revmean, [1.0, 1.0, 1.0])
revNormalize = TF.Compose([revnormalize1, revnormalize2])

revtrans = TF.Compose([revnormalize1, revnormalize2, TF.ToPILImage()])



In [4]:
# testset = datas.AIMSequence(config.testset_root, trans, config.test_size, config.test_crop_size, config.inter_frames)
# sampler = torch.utils.data.SequentialSampler(testset)
# validationloader = torch.utils.data.DataLoader(testset, sampler=sampler, batch_size=1, shuffle=False, num_workers=1)


# model
model = getattr(models, config.model)(config.pwc_path).cuda()
model = nn.DataParallel(model)

tot_time = 0
tot_frames = 0

print('Everything prepared. Ready to test...')

Everything prepared. Ready to test...


In [5]:
I_ = []

for i in range(5):
    filepath = f'datasets/example/car-turn/0000{i}.jpg'
    I = torch.Tensor(imread(filepath))
    I = I.permute(dims=(2,0,1)).unsqueeze(0)
    I_.append(I)
    


In [7]:
result = model(I_[0], I_[1], I_[3], I_[4], 0.5)

c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_corners=False from 1.4.0. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior will be changed "
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 8.00 GiB total capacity; 4.05 GiB already allocated; 0 bytes free; 115.21 MiB cached)